In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from pprint import pprint
import pandas as pd
import simuran
from simuran.loaders.allen_loader import AllenVisualBehaviorLoader
from simuran.loaders.one_loader import OneAlyxLoader
from simuran.loaders.nwb_loader import NWBLoader

In [ ]:
ALLEN_CACHE_DIR = Path(r"D:\example-data\allen-data")
ALLEN_MANIFEST = "visual-behavior-neuropixels_project_manifest_v0.4.0.json"
IBL_CACHE_DIR = Path(r"D:\example-data\ibl-data")

In [ ]:
allen_loader = AllenVisualBehaviorLoader(
    cache_directory=ALLEN_CACHE_DIR, manifest=ALLEN_MANIFEST
)
allen_sessions = allen_loader.get_sessions_table()
one_loader = OneAlyxLoader.from_cache(IBL_CACHE_DIR)
one_sessions = one_loader.get_sessions_table()

In [ ]:
allen_recording_container = simuran.RecordingContainer.from_table(allen_sessions, allen_loader)
one_recording_container = simuran.RecordingContainer.from_table(one_sessions, one_loader)

In [ ]:
from simuran.plot.unit import plot_unit_properties
from simuran.bridges.allen_vbn_bridge import (
    filter_good_units,
    brain_regions_in_structure,
)

output_dir = Path(r"D:\OpenDataResults")

all_units = allen_loader.get_units()
good_units = filter_good_units(all_units, sort_=False).copy()
for name, unit in zip(("all", "good"), (all_units, good_units)):
    plot_unit_properties(
        unit,
        ["firing_rate"],
        log_scale=[True],
        output_directory=output_dir / f"{name}_units_properties",
        region_dict=brain_regions_in_structure,
        split_regions=True,
    )

In [ ]:
# allen_recording_container.table.to_csv("allen_recordings.csv")
# one_recording_container.table.to_csv("one_recordings.csv")

In [ ]:
id_name = allen_recording_container[0].attrs["id_name"]
example_allen_recording = allen_recording_container.find_recording_by_attribute(
    id_name, 1044385384
)
example_allen_recording.load()

In [ ]:
nwb_loader = NWBLoader()
nwb_source_file = allen_loader.path_to_nwb(example_allen_recording)
nwb_allen_recording = simuran.Recording(source_file=nwb_source_file, loader=nwb_loader)
nwb_allen_recording.load()

In [ ]:
eid = one_loader.find_eid("steinmetzlab", "NR_0017")[0][0]
example_one_recording = one_recording_container.find_recording_by_attribute(
    "session", eid
)
example_one_recording.load()

In [ ]:
from brainbox.behavior.training import get_signed_contrast

def extract_useful_one(recording):
    result_dict = {}
    trials = recording.data["trials"]
    
    # https://int-brain-lab.github.io/iblenv/notebooks_external/loading_trials_data.html
    result_dict["trial_contrasts"] = get_signed_contrast(trials)
    result_dict["trial_correct"] = trials["feedbackType"] 

    trial_starts = trials["stimOn_times"]
    trial_ends = trials["response_times"]
    result_dict["trial_times"] = [(x, y) for x, y in zip(trial_starts, trial_ends)]

    spike_train = recording.data["probe00"]

    return result_dict

In [ ]:
from simple_analysis import filter_good_one_units
info = extract_useful_one(example_one_recording)
print(info["trial_times"][1])
good_units = filter_good_one_units(example_one_recording)

In [ ]:
from simple_analysis import create_spike_train_one
spike_train = create_spike_train_one(example_one_recording, good_units)

In [ ]:
from simple_analysis import simple_behaviour_compare
res = simple_behaviour_compare(spike_train["probe00"], info["trial_correct"], info["trial_times"])
res.to_csv("one_result.csv")

In [ ]:
from simple_analysis import extract_useful_allen
trial_times, good_units, spike_times, passed = extract_useful_allen(
    example_allen_recording)
good_units.to_csv("good_units_allen.csv")

In [ ]:
res = simple_behaviour_compare(spike_times, passed, trial_times)
res.to_csv("allen_result.csv")

In [ ]:
help(example_allen_recording.data)

In [ ]:
simple_behaviour_compare(spike_times, )

In [ ]:
summary = example_one_recording.summarise()

In [ ]:
trials_table = example_one_recording.data["trials"]
for k, v in example_one_recording.data.items():
    print(k, v)

In [ ]:
print(info)

In [ ]:
def extract_useful_allen(recording):
    def filter_units(unit_channels):
        unit_channels = unit_channels.sort_values(
            "probe_vertical_position", ascending=False
        )
        good_unit_filter = (
            (unit_channels["snr"] > 1)
            & (unit_channels["isi_violations"] < 1)
            & (unit_channels["firing_rate"] > 0.01)
            & (unit_channels["quality"])
        )
        return unit_channels[good_unit_filter]

    session = recording.data
    units = session.get_units()
    channels = session.get_channels()
    unit_channels = units.merge(channels, left_on="peak_channel_id", right_index=True)
    unit_channels.to_csv("units.csv")

    stimulus_presentations = session.stimulus_presentations
    change_times = stimulus_presentations[
        stimulus_presentations["active"] & stimulus_presentations["is_change"]
    ]["start_time"].values

    good_units = filter_units(unit_channels)

    return change_times, good_units, session.spike_times

In [ ]:
example_allen_recording.data.stimulus_presentations

In [ ]:
change_times, good_units, spike_times = extract_useful_allen(example_allen_recording)

In [ ]:
from analysis_functions import example_plot
fig = example_plot(good_units, spike_times, change_times)

In [ ]:
from analysis_functions import ccf_unit_plot
ccf_unit_plot(good_units)

In [ ]:
from analysis_functions import verify_filtering, filter_good_units, get_brain_regions_units
from matplotlib import pyplot as plt
all_units = allen_loader.cache.get_unit_table()
channels = allen_loader.cache.get_channel_table()
merged_units = all_units.merge(channels, left_on="ecephys_channel_id", right_index=True, suffixes=(None, "_y"))
merged_units.to_csv("test.csv")
good_units = filter_good_units(merged_units, sort_=False)

verify_filtering(good_units)
plt.show()
brain_regions = get_brain_regions_units(good_units)
print(brain_regions)